# Étape 1 : Préparer l’environnement

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os

Créez une session Spark pour démarrer PySpark :

In [3]:
# Créez une session Spark
spark = SparkSession.builder.appName("Manipulez des images avec PySpark") \
    .master("local[2]") \
    .getOrCreate()

25/01/20 23:05:35 WARN Utils: Your hostname, quillaur-Zen-AIO-24-ZN242GD-ZN242GD resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface eno2)
25/01/20 23:05:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/20 23:05:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Étape 2 : Charger des images de chats et de chiens

Le dataset Dogs vs Cats est disponible sur Kaggle : [Dogs vs Cats Dataset](https://www.kaggle.com/c/dogs-vs-cats/data).

Téléchargez et extrayez les images dans un dossier (par exemple, cats_dogs/).

**N'hésitez pas à réduire la taille du jeu de données** dans un premier temps. Quand on cuisine un code, on a pas besoin de tout le dataset ! Ce n'est qu'une fois que le code est bon qu'on test avec le dataset dans son ensemble. Dans mon cas, je n'ai conservé que 21 images par classe dans mon fichier cats_dogs_mini.zip.

Puisque j'ai uploadé un fichier zippé, je dois d'abord le dézipper.

In [8]:
%%capture
# !unzip cats_dogs_mini.zip

In [9]:
# Définissez le chemin vers votre dataset (à adapter selon votre structure de fichiers)
DATASET_PATH = "cats_dogs_mini"

Cette fonction charge une image, la redimensionne (pour correspondre à l’entrée du CNN), et la prétraite.

In [10]:
# Chargez les chemins des fichiers et les classes (cat/dog)
def extract_label(filepath):
    return "cat" if "cat" in filepath.split("/")[-1] else "dog"

def get_image_paths(directory):
    image_paths = []
    for root, dirs, files in os.walk(directory):
      for filename in files:
        if filename.endswith(".jpg"):
            image_paths.append(os.path.join(root, filename))
    return image_paths

## Calcul distribué :

Cette partie du code est prit en charge automtiquement par PySpark pour être distribuée si possible. En effet, le DataFrame Spark est réparti entre les nœuds du cluster.

Chaque nœud traite un sous-ensemble des données (par exemple, un lot d'images).

Chargez les images et leurs étiquettes ("chat" ou "chien") dans un DataFrame :

In [11]:
# Liste des chemins
image_paths = get_image_paths(DATASET_PATH)

# Créer un DataFrame Spark à partir des chemins
image_df = spark.createDataFrame([(path, extract_label(path)) for path in image_paths], ["path", "label"])

In [12]:
# Affichez quelques lignes du DataFrame
image_df.show(5, truncate=False)

+--------------------------------+-----+
|path                            |label|
+--------------------------------+-----+
|cats_dogs_mini/dogs/dog.4252.jpg|dog  |
|cats_dogs_mini/dogs/dog.4258.jpg|dog  |
|cats_dogs_mini/dogs/dog.4269.jpg|dog  |
|cats_dogs_mini/dogs/dog.4253.jpg|dog  |
|cats_dogs_mini/dogs/dog.4265.jpg|dog  |
+--------------------------------+-----+
only showing top 5 rows



## Calcul distribué :
Les opérations comme `groupBy` sont distribuées, chaque partition effectuant un traitement local avant agrégation globale.

In [13]:
# Comptez le nombre d'images par classe
image_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  dog|   21|
|  cat|   21|
+-----+-----+



# Étape 3 : Catégorisation automatique

In [14]:
# Réalisez une classification simple avec les caractéristiques extraites.
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

In [15]:
# Charger le modèle pré-entrainé
model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

/tmp/ipykernel_665594/4091702483.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")
W0000 00:00:1737414407.805976  665594 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [16]:
# Fonction pour extraire les caractéristiques d'une image
def extract_features(path):
    img = load_img(path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    features = model.predict(img_array)
    return Vectors.dense(features.flatten())

## Calcul distribué :
L'UDF `extract_features` est appliquée en parallèle sur chaque partition des données.

In [17]:
# Déclarez la fonction utilisateur (UDF)
from pyspark.sql.types import ArrayType, FloatType
# feature_udf = udf(extract_features, ArrayType(FloatType()))
feature_udf = udf(extract_features, VectorUDT())

In [18]:
# Ajoutez une colonne avec les caractéristiques extraites
image_df = image_df.withColumn("features", feature_udf(col("path")))

In [19]:
# Affichez le DataFrame avec les nouvelles colonnes
image_df.show(5)

2025-01-20 23:06:50.198154: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:06:50.201277: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:06:50.209359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737414410.223620  668767 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737414410.227906  668767 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 23:06:50.243065: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

+--------------------+-----+--------------------+
|                path|label|            features|
+--------------------+-----+--------------------+
|cats_dogs_mini/do...|  dog|[0.26830497384071...|
|cats_dogs_mini/do...|  dog|[1.00685632228851...|
|cats_dogs_mini/do...|  dog|[0.05627692490816...|
|cats_dogs_mini/do...|  dog|[0.0,0.0,1.145988...|
|cats_dogs_mini/do...|  dog|[0.06609643995761...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [20]:
# Indexer les labels pour la classification
from pyspark.ml.feature import StringIndexer
label_indexer = StringIndexer(inputCol="label", outputCol="label_index")
# data = label_indexer.fit(vectorized_df).transform(vectorized_df)
data = label_indexer.fit(image_df).transform(image_df)

In [21]:
data.show(5)

2025-01-20 23:06:56.085410: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:06:56.088398: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:06:56.096258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737414416.110535  669218 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737414416.114607  669218 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 23:06:56.129768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

+--------------------+-----+--------------------+-----------+
|                path|label|            features|label_index|
+--------------------+-----+--------------------+-----------+
|cats_dogs_mini/do...|  dog|[0.26830497384071...|        1.0|
|cats_dogs_mini/do...|  dog|[1.00685632228851...|        1.0|
|cats_dogs_mini/do...|  dog|[0.05627692490816...|        1.0|
|cats_dogs_mini/do...|  dog|[0.0,0.0,1.145988...|        1.0|
|cats_dogs_mini/do...|  dog|[0.06609643995761...|        1.0|
+--------------------+-----+--------------------+-----------+
only showing top 5 rows



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [22]:
# Diviser les données en ensembles d'entraînement et de test
train, test = data.randomSplit([0.8, 0.2], seed=42)

## Calcul distribué :
Spark ML divise les données d'entraînement entre les nœuds pour entraîner le modèle de manière parallèle.

In [23]:
%%time
# Entraîner un modèle de régression logistique
lr = LogisticRegression(featuresCol="features", labelCol="label_index")
model = lr.fit(train)

2025-01-20 23:07:02.810751: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:07:02.812840: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:07:02.814215: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:07:02.816255: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 23:07:02.824764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-20 23:07:02.826729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been

CPU times: user 54.4 ms, sys: 5.15 ms, total: 59.6 ms
Wall time: 14.2 s


In [24]:
%%time
# Évaluer le modèle sur les données de test
predictions = model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Précision du modèle : {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step 2) / 2]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 970ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/

Précision du modèle : 1.0
CPU times: user 26.9 ms, sys: 4.69 ms, total: 31.6 ms
Wall time: 4.03 s


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [25]:
# Arrêt de la session Spark
spark.stop()